# 错误处理

## **try...except...finally...**

用一个例子来看看 `try` 的机制：

当我们认为某些代码可能会出错时，就可以用 `try` 来运行这段代码，如果执行出错，则后续代码不会继续执行，而是直接跳转至错误处理代码，即except语句块，执行完`except` 后，如果有 `finally` 语句块，则执行 `finally` 语句块，至此，执行完毕。

In [24]:
try:
    print('try...')
    r = 10 / 0
    print('result:', r)
except ZeroDivisionError as e:
    print('except:', e)
finally:
    print('finally...')
print('END')

try...
except: division by zero
finally...
END


你还可以猜测，错误应该有很多种类，如果发生了不同类型的错误，应该由不同的 `except` 语句块处理。

没错，可以有多个 `except` 来捕获不同类型的错误：

In [25]:
try:
    print('try...')
    r = 10 / int('a')
    print('result:', r)
except ValueError as e:
    print('ValueError:', e)
except ZeroDivisionError as e:
    print('ZeroDivisionError:', e)
finally:
    print('finally...')
print('END')

try...
ValueError: invalid literal for int() with base 10: 'a'
finally...
END


## **except...else...**

此外，如果没有错误发生，可以在 `except` 语句块后面加一个 `else` ，当没有错误发生时，会自动执行 `else` 语句：

In [29]:
try:
    print('try...')
    r = 10 / int(1)
    print('result:', r)
except ValueError as e:
    print('ValueError:', e)
except ZeroDivisionError as e:
    print('ZeroDivisionError:', e)
else:
    print('no error! ' * 3)
finally:
    print('finally...')
print('END')

try...
result: 10.0
no error! no error! no error! 
finally...
END


 `Python` 的错误其实也是 `class` ，所有的错误类型都继承自 `BaseException` ，所以在使用 `except` 时需要注意的是，它不但捕获该类型的错误，还把其子类也“一网打尽”。比如：
 
```py
try:
    foo()
except ValueError as e:
    print('ValueError')
except UnicodeError as e:
    print('UnicodeError')
```

第二个 `except` 永远也捕获不到 `UnicodeError` ，因为 `UnicodeError` 是 `ValueError` 的子类，如果有，也被第一个 `except` 给捕获了。

`Python` 所有的错误都是从 `BaseException` 类派生的

## 多层调用

使用 `try...except` 捕获错误还有一个巨大的好处，就是可以跨越多层调用，比如函数 `main()调用foo()` ， `foo()调用bar()` ，结果 `bar()` 出错了，这时，只要 `main()` 捕获到了，就可以处理：


In [31]:

def foo(s):
    return 10 / int(s)

def bar(s):
    return foo(s) * 2

def main():
    try:
        bar('0')
    except Exception as e:
        print('Error:', e)
    finally:
        print('finally...')
        
        
main()

Error: division by zero
finally...


## 调用栈

如果错误没有被捕获，它就会一直往上抛，最后被 `Python` 解释器捕获，打印一个错误信息，然后程序退出。


In [32]:
# err.py:
def foo(s):
    return 10 / int(s)

def bar(s):
    return foo(s) * 2

def main():
    bar('0')

main()

ZeroDivisionError: division by zero

错误信息第1行：

    Traceback (most recent call last):

告诉我们这是错误的跟踪信息。

第2~3行：

    File "err.py", line 11, in <module>
        main()
        
调用 `main()` 出错了，在代码文件err.py的第11行代码，但原因是第9行：

      File "err.py", line 9, in main
        bar('0')
        
调用 `bar('0')` 出错了，在代码文件 `err.py` 的第9行代码，但原因是第6行：

    File "err.py", line 6, in bar
        return foo(s) * 2
        
原因是 `return foo(s) * 2` 这个语句出错了，但这还不是最终原因，继续往下看：

    File "err.py", line 3, in foo
        return 10 / int(s)
    
原因是 `return 10 / int(s)` 这个语句出错了，这是错误产生的源头，因为下面打印了：

    ZeroDivisionError: integer division or modulo by zero

## 记录错误[logging]

如果不捕获错误，自然可以让 `Python` 解释器来打印出错误堆栈，但程序也被结束了。

既然我们能捕获错误，就可以把错误堆栈打印出来，然后分析错误原因，同时，让**程序继续执行下去**。

`Python` 内置的 `logging` 模块可以非常容易地记录错误信息：

In [33]:
# err_logging.py

import logging

def foo(s):
    return 10 / int(s)

def bar(s):
    return foo(s) * 2

def main():
    try:
        bar('0')
    except Exception as e:
        logging.exception(e)

main()
print('END')

ERROR:root:division by zero
Traceback (most recent call last):
  File "<ipython-input-33-50dd10eca926>", line 13, in main
    bar('0')
  File "<ipython-input-33-50dd10eca926>", line 9, in bar
    return foo(s) * 2
  File "<ipython-input-33-50dd10eca926>", line 6, in foo
    return 10 / int(s)
ZeroDivisionError: division by zero


END


## 抛出错误[raise]

因为错误是 `class` ，捕获一个错误就是捕获到该 `class` 的一个实例。

因此，错误并不是凭空产生的，而是有意创建并抛出的。

`Python` 的内置函数会抛出很多类型的错误，我们自己编写的函数也可以抛出错误。

如果要抛出错误，首先根据需要，可以定义一个错误的 `class` ，选择好继承关系，然后，用 `raise` 语句抛出一个错误的实例：


In [34]:
# err_raise.py
class FooError(ValueError):
    pass

def foo(s):
    n = int(s)
    if n==0:
        raise FooError('invalid value: %s' % s)
    return 10 / n

foo('0')

FooError: invalid value: 0

只有在必要的时候才定义我们自己的错误类型。如果可以选择 `Python` 已有的内置的错误类型（比如 `ValueError` ， `TypeError` ），尽量使用 `Python` 内置的错误类型。

最后，我们来看另一种错误处理的方式：

在 `bar()` 函数中，我们明明已经捕获了错误，但是，打印一个 `ValueError!` 后，又把错误通过 `raise` 语句抛出去了，这不有病么？

其实这种错误处理方式不但没病，而且相当常见。

捕获错误目的只是记录一下，便于后续追踪。

但是，由于当前函数不知道应该怎么处理该错误，所以，最恰当的方式是继续往上抛，让顶层调用者去处理。

好比一个员工处理不了一个问题时，就把问题抛给他的老板，如果他的老板也处理不了，就一直往上抛，最终会抛给CEO去处理。

 `raise` 语句如果不带参数，就会把当前错误原样抛出。
 

In [35]:
# err_reraise.py

def foo(s):
    n = int(s)
    if n==0:
        raise ValueError('invalid value: %s' % s)
    return 10 / n

def bar():
    try:
        foo('0')
    except ValueError as e:
        print('ValueError!')
        raise

bar()

ValueError!


ValueError: invalid value: 0

此外，在 `except` 中 `raise` 一个 `Error` ，还可以把一种类型的错误转化成另一种类型：

In [36]:
try:
    10 / 0
except ZeroDivisionError:
    raise ValueError('input error!')

ValueError: input error!

In [38]:
from functools import reduce

def str2num(s):
    return float(s)

def calc(exp):
    ss = exp.split('+')
    ns = map(str2num, ss)
    return reduce(lambda acc, x: acc + x, ns)

def main():
    r = calc('100 + 200 + 345')
    print('100 + 200 + 345 =', r)
    r = calc('99 + 88 + 7.6')
    print('99 + 88 + 7.6 =', r)

main()

100 + 200 + 345 = 645.0
99 + 88 + 7.6 = 194.6


# 调试

## print()

In [39]:
def foo(s):
    n = int(s)
    print('>>> n = %d' % n)
    return 10 / n

def main():
    foo('0')

main()

>>> n = 0


ZeroDivisionError: division by zero

## 断言

凡是用 `print()` 来辅助查看的地方，都可以用断言（ `assert` ）来替代：

 `assert` 的意思是，表达式 `n != 0` 应该是 `True` ，否则，根据程序运行的逻辑，后面的代码肯定会出错。

如果断言失败， `assert` 语句本身就会抛出 `AssertionError` ：

In [46]:
def foo(s):
    n = int(s)
    assert n != 0, 'n is zero!'
    return 10 / n

def main():
    foo(0)
    
main()

AssertionError: n is zero!

In [44]:
%%file err.py

def foo(s):
    n = int(s)
    assert n != 0, 'n is zero!'
    return 10 / n

def main():
    foo(0)
    
main()

Writing err.py


启动 `Python` 解释器时可以用 `-O` 参数来关闭 `assert` ：

    python -O err.py


In [45]:
!python -O err.py

Traceback (most recent call last):
  File "err.py", line 10, in <module>
    main()
  File "err.py", line 8, in main
    foo(0)
  File "err.py", line 5, in foo
    return 10 / n
ZeroDivisionError: division by zero


## logging

把 `print()` 替换为 `logging` 是第3种方式，和 `assert` 比， `logging` 不会抛出错误，而且可以输出到文件：

In [48]:
import logging

# 初始化 logging
logging.basicConfig(level=logging.INFO)

s = '0'
n = int(s)
logging.info('n = %d' % n)
print(10 / n)

ZeroDivisionError: division by zero

## pdb... IDE... 斷點

.......

# 单元测试

**來源**

https://www.liaoxuefeng.com/wiki/1016959663602400/1017604210683936

以测试为驱动的开发模式最大的好处就是确保一个程序模块的行为符合我们设计的测试用例。

在将来修改的时候，可以极大程度地保证该模块行为仍然是正确的。

## 例子[mydict]

来编写一个 `Dict` 类，这个类的行为和 `dict` 一致，但是可以通过属性来访问，用起来就像下面这样：
```py
>>> d = Dict(a=1, b=2)
>>> d['a']
1
>>> d.a
1
```

In [1]:
%%file mydict.py

class Dict(dict):

    def __init__(self, **kw):
        super().__init__(**kw)

    def __getattr__(self, key):
        try:
            return self[key]
        except KeyError:
            raise AttributeError(r"'Dict' object has no attribute '%s'" % key)

    def __setattr__(self, key, value):
        self[key] = value

Writing mydict.py


### unittest

为了编写单元测试，我们需要引入 `Python` 自带的 `unittest` 模块，编写 `mydict_test.py` 如下：

编写单元测试时，我们需要编写一个测试类，从 `unittest.TestCase` 继承。

以 `test` 开头的方法就是测试方法，不以 `test` 开头的方法不被认为是测试方法，测试的时候不会被执行。

对每一类测试都需要编写一个 `test_xxx()` 方法。

由于 `unittest.TestCase` 提供了很多内置的条件判断，我们只需要调用这些方法就可以断言输出是否是我们所期望的。

最常用的断言就是 `assertEqual()` ：

```py
self.assertEqual(abs(-1), 1) # 断言函数返回的结果与1相等
```

另一种重要的断言就是期待抛出指定类型 `Error` ，比如通过 `d['empty']` 访问不存在的 `key` 时，断言会抛出 `KeyError` ：

```py
with self.assertRaises(KeyError):
    value = d['empty']
```

而通过 `d.empty` 访问不存在的 `key` 时，我们期待抛出 `AttributeError` ：

```py
with self.assertRaises(AttributeError):
    value = d.empty
```

In [20]:
%%file mydict_test.py

import unittest

from mydict import Dict

class TestDict(unittest.TestCase):

    def test_init(self):
        d = Dict(a=1, b='test')
        self.assertEqual(d.a, 1)
        self.assertEqual(d.b, 'test')
        self.assertTrue(isinstance(d, dict))

    def test_key(self):
        d = Dict()
        d['key'] = 'value'
        self.assertEqual(d.key, 'value')

    def test_attr(self):
        d = Dict()
        d.key = 'value'
        self.assertTrue('key' in d)
        self.assertEqual(d['key'], 'value')

    def test_keyerror(self):
        d = Dict()
        with self.assertRaises(KeyError):
            value = d['empty']

    def test_attrerror(self):
        d = Dict()
        with self.assertRaises(AttributeError):
            value = d.empty
            
# 若無 if __name__ == '__main__': 則使用 
# !python -m unittest mydict_test
# 可批量运行很多单元测试
if __name__ == '__main__':
    unittest.main()

Overwriting mydict_test.py


### 開啟測試方式

In [11]:
# 直接用 python 開啟

!python mydict_test.py

.....
----------------------------------------------------------------------
Ran 5 tests in 0.001s

OK


In [21]:
# 用 unittest 開啟

# 若無 if __name__ == '__main__': 則使用 
# !python -m unittest mydict_test
# 可批量运行很多单元测试

!python -m unittest mydict_test

.....
----------------------------------------------------------------------
Ran 5 tests in 0.001s

OK


### setUp与tearDown

可以在单元测试中编写两个特殊的 `setUp()` 和 `tearDown()` 方法。这两个方法会分别在每调用一个测试方法的前后分别被执行。

`setUp()` 和 `tearDown()` 方法有什么用呢？设想你的测试需要启动一个数据库，这时，就可以在 `setUp()` 方法中连接数据库，在 `tearDown()` 方法中关闭数据库，这样，不必在每个测试方法中重复相同的代码：

```py
class TestDict(unittest.TestCase):

    def setUp(self):
        print('setUp...')

    def tearDown(self):
        print('tearDown...')
```        
        
可以再次运行测试看看每个测试方法调用前后是否会打印出 `setUp...` 和`tearDown...`


## 练习

对 `Student` 类编写单元测试，结果发现测试不通过，请修改 `Student` 类，

让测试通过：

In [16]:
%%file Student.py
class Student(object):
    def __init__(self, name, score):
        self.name = name
        self.score = score
    def get_grade(self):
        if self.score >= 60:
            return 'B'
        if self.score >= 80:
            return 'A'
        return 'C'

Writing Student.py


In [18]:
%%file Student_test.py

import unittest
from Student import Student

class TestStudent(unittest.TestCase):

    def test_80_to_100(self):
        s1 = Student('Bart', 80)
        s2 = Student('Lisa', 100)
        self.assertEqual(s1.get_grade(), 'A')
        self.assertEqual(s2.get_grade(), 'A')

    def test_60_to_80(self):
        s1 = Student('Bart', 60)
        s2 = Student('Lisa', 79)
        self.assertEqual(s1.get_grade(), 'B')
        self.assertEqual(s2.get_grade(), 'B')

    def test_0_to_60(self):
        s1 = Student('Bart', 0)
        s2 = Student('Lisa', 59)
        self.assertEqual(s1.get_grade(), 'C')
        self.assertEqual(s2.get_grade(), 'C')

    def test_invalid(self):
        s1 = Student('Bart', -1)
        s2 = Student('Lisa', 101)
        with self.assertRaises(ValueError):
            s1.get_grade()
        with self.assertRaises(ValueError):
            s2.get_grade()

if __name__ == '__main__':
    
    unittest.main()

Overwriting Student_test.py


In [19]:
!python Student_test.py

..FF
FAIL: test_80_to_100 (__main__.TestStudent)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "Student_test.py", line 10, in test_80_to_100
    self.assertEqual(s1.get_grade(), 'A')
AssertionError: 'B' != 'A'
- B
+ A


FAIL: test_invalid (__main__.TestStudent)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "Student_test.py", line 29, in test_invalid
    s1.get_grade()
AssertionError: ValueError not raised

----------------------------------------------------------------------
Ran 4 tests in 0.001s

FAILED (failures=2)


In [22]:
%%file Student.py

class Student(object):
    def __init__(self, name, score):
        self.name = name
        self.score = score
    def get_grade(self):
        if self.score > 100:
            raise ValueError
        elif self.score >= 80:
            return 'A'
        elif self.score >= 60:
            return 'B'
        elif self.score >= 0:
            return 'C'
        else:
            raise ValueError

Overwriting Student.py


In [23]:
!python Student_test.py

....
----------------------------------------------------------------------
Ran 4 tests in 0.000s

OK
